This notebook loads the model from a h5 file and generates confusion matrix. 

In [1]:
from keras.models import load_model
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras import backend
from keras.callbacks import ModelCheckpoint
from numpy import unique, ndarray, mean, std
from matplotlib import pyplot as plt
from pandas import read_pickle
import numpy as np 
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
import itertools

Using TensorFlow backend.


In [2]:
# batch_size = 100
# epochs = 20
vector_encoding = True
img_height, img_width = 48, 48

In [3]:
filepath = "adam-data-aug.h5"
model = load_model(filepath)

In [4]:
def normalize(x):
    x = x - mean(x, axis = 1).reshape((-1,1))
    x = (x - mean(x, axis = 0)) / std(x, axis = 0)
    return x

In [5]:
data = read_pickle('dataset/input_data')

train = data.Training
validate = data.Validation
test = data.Testing

x_train, y_train = train
x_validate, y_validate = validate
x_test, y_test = test

x_train = normalize(x_train)
x_validate = normalize(x_validate)
x_test = normalize(x_test)

num_train_samples = x_train.shape[0]
num_validate_samples = x_validate.shape[0]
num_test_samples = x_test.shape[0]
num_labels = unique(y_train).shape[0]
if(vector_encoding):
    y_train = keras.utils.to_categorical(y_train, num_labels)
    y_validate = keras.utils.to_categorical(y_validate, num_labels)
    y_test = keras.utils.to_categorical(y_test, num_labels)
    
if backend.image_data_format() == 'channels_first':
    x_train = x_train.reshape(num_train_samples, 1, img_height, img_width)
    x_validate = x_validate.reshape(num_validate_samples, 1, img_height, img_width)
    x_test = x_test.reshape(num_test_samples, 1, img_height, img_width)
    input_shape = (1, img_height, img_width)
else:
    x_train = x_train.reshape(num_train_samples, img_height, img_width, 1)
    x_validate = x_validate.reshape(num_validate_samples, img_height, img_width, 1)
    x_test = x_test.reshape(num_test_samples, img_height, img_width, 1)
    input_shape = (img_height, img_width, 1)

In [6]:
# model.summary()

In [7]:
# Source - from the library's website
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


In [8]:
y_pred = model.predict(x_test)

In [ ]:
model.evaluate(x_test, y_test, verbose=1)

In [10]:
y_actual_cnf = np.argmax(y_test,axis=1)
y_pred_cnf = np.argmax(y_pred,axis=1)

In [11]:
cnf_matrix = confusion_matrix(y_actual_cnf,y_pred_cnf)

In [ ]:
class_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix with Data Augmentation')